In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load dataset
file_path =  r"C:\Users\user\Downloads\Apple\aapl_us_2025.csv"

df = pd.read_csv(file_path)

# Handle missing values
df.dropna(inplace=True)

# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df.dropna(subset=['Date'], inplace=True)

# Ensure 'Close' column is numeric
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
df.dropna(subset=['Close'], inplace=True)

# Use only the 'Close' column for prediction
data = df[['Close']].values

# Normalize data
scaler = MinMaxScaler(feature_range=(0,1))

train_size = int(len(data) * 0.8)
train_data, test_data = data[:train_size], data[train_size:]

train_scaled = scaler.fit_transform(train_data)  # Fit on train data
test_scaled = scaler.transform(test_data)  # Transform test data

# Function to create sequences
def create_sequences(data, seq_length=50):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Create sequences
seq_length = 50
X_train, y_train = create_sequences(train_scaled, seq_length)
X_test, y_test = create_sequences(test_scaled, seq_length)

# Reshape for LSTM/GRU input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Function to create and train model
def build_and_train_model(model_type='LSTM'):
    model = Sequential()
    if model_type == 'LSTM':
        model.add(LSTM(100, return_sequences=True, input_shape=(seq_length, 1)))
        model.add(Dropout(0.2))
        model.add(LSTM(100, return_sequences=False))
    else:  # GRU
        model.add(GRU(100, return_sequences=True, input_shape=(seq_length, 1)))
        model.add(Dropout(0.2))
        model.add(GRU(100, return_sequences=False))
    
    model.add(Dropout(0.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stop], verbose=1)
    
    return model, history

# Train LSTM and GRU models
lstm_model, lstm_history = build_and_train_model('LSTM')
gru_model, gru_history = build_and_train_model('GRU')

# Predict
y_pred_lstm = lstm_model.predict(X_test)
y_pred_gru = gru_model.predict(X_test)

# Convert back to original scale
y_pred_lstm = scaler.inverse_transform(y_pred_lstm)
y_pred_gru = scaler.inverse_transform(y_pred_gru)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1,1))

# Evaluation
def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f"{model_name} - MSE: {mse:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")

evaluate_model(y_test_actual, y_pred_lstm, "LSTM")
evaluate_model(y_test_actual, y_pred_gru, "GRU")

# Plot results
plt.figure(figsize=(12,6))
plt.plot(df['Date'][train_size+seq_length:], y_test_actual, label='Actual Price', color='blue')
plt.plot(df['Date'][train_size+seq_length:], y_pred_lstm, label='LSTM Prediction', color='red')
plt.plot(df['Date'][train_size+seq_length:], y_pred_gru, label='GRU Prediction', color='green')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.title('Apple Stock Price Prediction using LSTM & GRU')
plt.legend()
plt.grid()
plt.show()


Epoch 1/50


c:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


253/253 ━━━━━━━━━━━━━━━━━━━━ 37s 120ms/step - loss: 0.0069 - val_loss: 5.7174
Epoch 2/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 43s 128ms/step - loss: 6.8334e-04 - val_loss: 5.3625
Epoch 3/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 41s 126ms/step - loss: 4.7398e-04 - val_loss: 4.2242
Epoch 4/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 29s 115ms/step - loss: 4.4556e-04 - val_loss: 4.3482
Epoch 5/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 34s 132ms/step - loss: 4.0047e-04 - val_loss: 3.9326
Epoch 6/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 34s 133ms/step - loss: 3.9189e-04 - val_loss: 3.9855
Epoch 7/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 34s 133ms/step - loss: 3.3629e-04 - val_loss: 3.1073
Epoch 8/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 36s 110ms/step - loss: 4.6962e-04 - val_loss: 3.3008
Epoch 9/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 40s 105ms/step - loss: 3.1166e-04 - val_loss: 3.0671
Epoch 10/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 27s 107ms/step - loss: 3.4274e-04 - val_loss: 3.2543
Epoch 11/50
253/253 ━━━━━━━━━━━━━━━━━━━━ 49s 136ms/step - loss: 2.8822e-04 - val_los